<a href="https://colab.research.google.com/github/AnanthSeetharaman/DataIngestionFrmwrkUsingADF/blob/master/RAG_Tutorial_4_OD_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain_openai

In [2]:
!apt-get update
!apt-get install -y postgresql-14-pgvector

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [3]:
%%sh
apt-get -qq install postgresql-server-dev-14

cd /tmp
rm -rf pgvector


git clone --branch v0.8.1 https://github.com/pgvector/pgvector.git

cd pgvector

make

make install # may need sudo




gcc -Wall -Wmissing-prototypes -Wpointer-arith -Wdeclaration-after-statement -Werror=vla -Wendif-labels -Wmissing-format-attribute -Wimplicit-fallthrough=3 -Wcast-function-type -Wformat-security -fno-strict-aliasing -fwrapv -fexcess-precision=standard -Wno-format-truncation -Wno-stringop-truncation -g -g -O2 -flto=auto -ffat-lto-objects -flto=auto -ffat-lto-objects -fstack-protector-strong -Wformat -Werror=format-security -fno-omit-frame-pointer -march=native -ftree-vectorize -fassociative-math -fno-signed-zeros -fno-trapping-math -fPIC -I. -I./ -I/usr/include/postgresql/14/server -I/usr/include/postgresql/internal -Wdate-time -D_FORTIFY_SOURCE=2 -D_GNU_SOURCE  -I/usr/include/libxml2   -c -o src/bitutils.o src/bitutils.c
gcc -Wall -Wmissing-prototypes -Wpointer-arith -Wdeclaration-after-statement -Werror=vla -Wendif-labels -Wmissing-format-attribute -Wimplicit-fallthrough=3 -Wcast-function-type -Wformat-security -fno-strict-aliasing -fwrapv -fexcess-precision=standard -Wno-format-trunc

Cloning into 'pgvector'...
Note: switching to '778dacf20c07caf904557a88705142631818d8cb'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false



In [4]:
# Install PostgreSQL
!apt-get update
!apt-get install -y postgresql postgresql-contrib

# Start PostgreSQL service
!service postgresql start

# Create a new user and database
import getpass
import os

user = "pwed"
password = getpass.getpass("Enter password for new PostgreSQL user '{}': ".format(user))
# Set the password as an environment variable for the psql command
os.environ['PGPASSWORD'] = password

# Create user and database using psql
!sudo -u postgres psql -c "CREATE USER {user} WITH PASSWORD '{password}';"
!sudo -u postgres psql -c "CREATE DATABASE test_db OWNER {user};"


# Test the connection
import psycopg2

try:
    conn = psycopg2.connect(dbname="test_db", user=user, password=password, host="localhost")
    print("Connection to PostgreSQL successful!")
    conn.close()
except psycopg2.OperationalError as e:
    print(f"Error connecting to PostgreSQL: {e}")

# Unset the password environment variable
del os.environ['PGPASSWORD']

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [5]:
%%sh

pg_ctlcluster 14 main start
sudo -u postgres psql -c "select version();"



Cluster is already running.
                                                                 version                                                                  
------------------------------------------------------------------------------------------------------------------------------------------
 PostgreSQL 14.19 (Ubuntu 14.19-0ubuntu0.22.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.4.0-1ubuntu1~22.04.2) 11.4.0, 64-bit
(1 row)



In [6]:
!sudo -u postgres psql -d test_db -c "GRANT USAGE ON SCHEMA public TO pwed;"

!sudo -u postgres psql -d test_db -c "CREATE EXTENSION IF NOT EXISTS vector;"
!sudo -u postgres psql -d test_db -c "GRANT USAGE ON SCHEMA public TO pwed;"


GRANT
NOTICE:  extension "vector" already exists, skipping
CREATE EXTENSION
GRANT


In [7]:
!sudo -u postgres psql -d test_db -c "CREATE EXTENSION IF NOT EXISTS vector;"


NOTICE:  extension "vector" already exists, skipping
CREATE EXTENSION


In [8]:
import psycopg2

user = "pwed"
password = getpass.getpass("Enter password for PostgreSQL user '{}': ".format(user))
db_name = "test_db"
host = "localhost"

try:
    conn = psycopg2.connect(dbname=db_name, user=user, password=password, host=host)
    conn.set_session(autocommit=True)
    print("Connection to PostgreSQL successful!")

    # Install pgvector extension
    with conn.cursor() as cur:
        cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
        print("pgvector extension created.")

    # Verify extension installation
    with conn.cursor() as cur:
        cur.execute("SELECT * FROM pg_extension WHERE extname = 'vector';")
        extension_status = cur.fetchone()
        if extension_status:
            print("pgvector extension is installed and active.")
        else:
            print("pgvector extension not found.")

    conn.close()

except psycopg2.OperationalError as e:
    print(f"Error connecting to PostgreSQL or executing commands: {e}")

Enter password for PostgreSQL user 'pwed': ··········
Connection to PostgreSQL successful!
pgvector extension created.
pgvector extension is installed and active.


In [11]:
import psycopg2
import psycopg2, numpy as np

user = "pwed"
password = getpass.getpass("Enter password for PostgreSQL user '{}': ".format(user))
db_name = "test_db"
host = "localhost"

try:
    conn = psycopg2.connect(dbname=db_name, user=user, password=password, host=host)
    conn.set_session(autocommit=True)
    print("Connection to PostgreSQL successful!")

    # Install pgvector extension
    with conn.cursor() as cur:
        cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")

        cur.execute("create table if not exists docs(id serial primary key, content text, embedding vector(384));")
        cur.execute("truncate table docs;")
        cur.execute("insert into docs(content,embedding) values (%s,%s),(%s,%s),(%s,%s)",
                    ('cat','[0.1,0.2,0.3]',
                     'dog','[0.11,0.19,0.29]',
                     'car','[0.9,0.8,0.7]'))

        q=np.array([0.1,0.21,0.31])
        cur.execute("select content, embedding <-> %s d from docs order by d limit 1;",(f"[{q[0]},{q[1]},{q[2]}]",))
        print(cur.fetchall())
        cur.close();

    conn.close()

except psycopg2.OperationalError as e:
    print(f"Error connecting to PostgreSQL or executing commands: {e}")

Enter password for PostgreSQL user 'pwed': ··········
Connection to PostgreSQL successful!


DataException: expected 384 dimensions, not 3


Core Semantic Work !!


In [12]:
%%bash
pip install sentence_transformers psycopg2-binary > /dev/null


In [13]:
import numpy as np
import pandas as pd
import requests
from os import mkdir, getcwd, chdir, listdir
from os.path import join
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
import pkg_resources
from matplotlib import pyplot as plt
from zipfile import ZipFile
from textwrap import wrap

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/tmp/ipython-input-3291340494.py:9: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [14]:
HOME = getcwd()

In [15]:
if 'catalogue' not in listdir():
    mkdir('catalogue')

if 'test.csv' not in listdir(join(HOME,'catalogue')):
    URL = "https://github.com/fabio-a-oliveira/semantic-search/blob/main/data/project_gutenberg_corpus___2021_05_01.csv?raw=true"
    URL = "https://github.com/pinecone-io/semantic-search-example/raw/refs/heads/main/test.csv"
    content = requests.get(URL).content
    with open(join(HOME,'catalogue','test.csv'), 'wb') as file:
        file.write(content)

catalogue = pd.read_csv(join(HOME, 'catalogue', 'test.csv'),
                        sep = ",")

In [16]:
catalogue

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?
...,...,...,...
99994,99994,What is the most overrated pleasure? Why?,What is the most pleasurable job in the world?
99995,99995,Which is the best open source tool to manage a...,Which tool to manage a Mass Open Online Course...
99996,99996,What could be the stomach between I don't and ...,"What's the difference good ™, ® and ©?"
99997,99997,Does the Facebook Public Feed API include the ...,Is there an API or data feed to find people's ...


In [17]:
import pandas as pd

df=pd.DataFrame({
"id":range(1,6),
"text":[
"How do I learn Python quickly?",
"What is the healthiest breakfast?",
"How to improve memory?",
"Best way to start investing?",
"What is machine learning?"
]
})
df.to_csv("/content/catalogue/qa.csv",index=False)
print(df)


   id                               text
0   1     How do I learn Python quickly?
1   2  What is the healthiest breakfast?
2   3             How to improve memory?
3   4       Best way to start investing?
4   5          What is machine learning?


In [18]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import psycopg2, getpass, numpy as np

df=pd.read_csv("/content/catalogue/qa.csv")
m=SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
E=m.encode(df["text"].tolist()).astype(float)

u=input()
p=getpass.getpass()
c=psycopg2.connect(dbname="test_db",user=u,password=p,host="localhost")
c.set_session(autocommit=True)
cur=c.cursor()

cur.execute("drop table if exists docs;")
dim=E.shape[1]
cur.execute(f"create table docs(id serial primary key,content text,embedding vector({dim}));")

for t,e in zip(df["text"],E):
    cur.execute("insert into docs(content,embedding) values (%s,%s)",(t,[float(x) for x in e]))

cur.close(); c.close()


pwed
··········


In [19]:
from sentence_transformers import SentenceTransformer
import psycopg2, getpass
import numpy as np

q="about memory"
m=SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
v=m.encode([q])[0]
u=input(); p=getpass.getpass()
c=psycopg2.connect(dbname="test_db",user=u,password=p,host="localhost")
cur=c.cursor()
# cur.execute("select content,embedding <-> %s d from docs order by d limit 3;",([float(x) for x in v],))

cur.execute(
    "select content, embedding <-> %s::vector d from docs order by d limit 3;",
    ("["+",".join(str(float(x)) for x in v)+"]",)
)


print(cur.fetchall())
cur.close(); c.close()


pwed
··········
[('How to improve memory?', 0.7797929910303221), ('What is machine learning?', 1.2220117009565745), ('How do I learn Python quickly?', 1.2605260638637141)]
